<a href="https://colab.research.google.com/github/medha-sagar/CSE547-BigDataForMachineLearning/blob/master/CSE547_Q2_correct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1rqNLakXn3uthZtXvHSzD6s2xn-w0NscJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('browsing.txt')

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter
%matplotlib inline

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Read the file and split the data. Also remove spaces

In [ ]:
browsing_data = spark.createDataFrame(
    sc.textFile("browsing.txt").map(lambda l: l.split('\n')),
    ["basket"]
)
items_in_browsing_data = sc.textFile("browsing.txt").flatMap(lambda line: line.split(" "))
items_in_browsing_data = items_in_browsing_data.filter(lambda x: x != "")


In [ ]:
items_in_browsing_data.take(5)

['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222']

Count the number if items which occur more than 100 times in all baskets

In [ ]:
itemCount = items_in_browsing_data.map(lambda item: (item, 1)).reduceByKey(lambda a,b:a +b)
itemCount = itemCount.filter(
    lambda x: x[1]>=100
)


In [ ]:
items_over_100 = itemCount.keys().collect()

In [ ]:
len(items_over_100)

647

Generate pairs out of the frequent single frequent items 

In [ ]:
pairs = list()
for subset in combinations(items_over_100, 2):
  subset = list(subset)
  subset.sort()
  pairs.append(tuple(subset))
freq_pairs_of_items = pairs
freq_pairs_of_items = dict(Counter(freq_pairs_of_items))

Get count of each of the pair in the baskets

In [ ]:
basket_pairs = {}
all_baskets = sc.textFile("browsing.txt").collect()
for basket in all_baskets:
  basket_list = basket.split(" ")
  basket_list.sort()
  basket_list.remove('')
  all_combinations = set(combinations(basket_list, 2))
  for comb in all_combinations:
    comb = list(comb)
    comb.sort()
    #if comb in pair_freq:
    if pair_freq.get(tuple(comb)) != None: 
      if tuple(comb) not in basket_pairs.keys():
        basket_pairs[tuple(comb)] = 1
      else:
        basket_pairs[tuple(comb)] += 1

Filter the pairs which have counts more than 100.

In [ ]:
basket_pairs = {k:v for (k,v) in basket_pairs.items() if v >= 100}

In [ ]:
print(len(basket_pairs))

1334


Calculate the confidence of each association

In [ ]:
confidences=[]
itemCount_df=itemCount.collect()
itemCount_df=pd.DataFrame(itemCount_df,columns=['item','cnt'])
for item in basket_pairs:
  conf_A_to_B=(basket_pairs[tuple([item[0], item[1]])]/(itemCount_df.loc[itemCount_df['item'] == item[0], 'cnt'].iloc[0]))
  confidences.append([item[0], item[1],conf_A_to_B])
  conf_B_to_A=(basket_pairs[tuple([item[0], item[1]])]/(itemCount_df.loc[itemCount_df['item'] == item[1], 'cnt'].iloc[0]))
  confidences.append([item[1], item[0],conf_B_to_A])
confidences_df=pd.DataFrame(confidences,columns=['A','B',"confidence"])
confidences_df= confidences_df.sort_values("confidence",ascending=False)
print(confidences_df[0:5])


             A         B  confidence
1258  DAI93865  FRO40251    1.000000
267   GRO85051  FRO40251    0.999176
251   GRO38636  FRO40251    0.990654
1182  ELE12951  FRO40251    0.990566
416   DAI88079  FRO40251    0.986726


Generate the triplets out of the frequent pairs. Generate the triplets by self joining and checking whether the first element of the 2 pairs are equal.

In [ ]:
freq_triplets_of_items = list()
for i in basket_pairs:
  for j in basket_pairs:
    tri=set(tuple([i[0],i[1],j[1]]))
    if i[0]==j[0] and len(tri)==3 :
      subset = list(tri)
      subset.sort()
      freq_triplets_of_items.append(tuple(subset))
freq_triplets_of_items = dict(Counter(triplet_freq))


In [ ]:
print(len(triplet_freq))

53716


Finding the count of each triplet in all the baskets

In [ ]:
basket_triplets = {}
for basket in all_baskets:
  basket_list = basket.split(" ")
  basket_list.remove('')
  basket_list.sort()
  all_combinations = set(combinations(basket_list, 3))
  for comb in all_combinations:
    comb = list(comb)
    comb.sort()
    if freq_triplets_of_items.get(tuple(comb)) != None: 
      if tuple(comb) not in basket_triplets.keys():
        basket_triplets[tuple(comb)] = 1
      else:
        basket_triplets[tuple(comb)] += 1


In [ ]:
len(basket_triplets)

26143

Filter the triplets whose count is less than 100 in the baskets

In [ ]:
basket_triplets = {k:v for (k,v) in basket_triplets.items() if v >= 100}

Calculating the confidences of triplets

In [ ]:
confidences=[]
itemCount_df=itemCount.collect()
itemCount_df=pd.DataFrame(itemCount_df,columns=['item','cnt'])
for item in basket_triplets:
  conf_AB_to_C=(basket_triplets[tuple([item[0], item[1], item[2]])]/(basket_pairs[tuple([item[0], item[1]])]))
  confidences.append([[item[0], item[1]],item[2],conf_AB_to_C])
  conf_BC_to_A=(basket_triplets[tuple([item[0], item[1], item[2]])]/(basket_pairs[tuple([item[1], item[2]])]))
  confidences.append([[item[1], item[2]],item[0],conf_BC_to_A])
  conf_CA_to_B=(basket_triplets[tuple([item[0], item[1], item[2]])]/(basket_pairs[tuple([item[0], item[2]])]))
  confidences.append([[item[0], item[2]],item[1],conf_CA_to_B])
confidences_df=pd.DataFrame(confidences,columns=['A','B',"confidence"])
confidences_df[["A1","A2"]] = pd.DataFrame(confidences_df.A.values.tolist(), index= confidences_df.index)
confidences_df=confidences_df.sort_values(by=["confidence", 'A1','A2','B'],ascending=[False, True, True,True])
confidences_df[0:5][['A','B','confidence']]

,A,B,confidence
689,"[DAI23334, ELE92920]",DAI62779,1.0
413,"[DAI31081, GRO85051]",FRO40251,1.0
215,"[DAI55911, GRO85051]",FRO40251,1.0
57,"[DAI62779, DAI88079]",FRO40251,1.0
164,"[DAI75645, GRO85051]",FRO40251,1.0
